In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

sns.set(font_scale=2.2)
#matplotlib의 stylesheet를 seaborn으로 지정
plt.style.use('seaborn')

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split, ShuffleSplit
from sklearn.metrics import f1_score
import itertools
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
# 
import shap
#
from tqdm import tqdm
import featuretools as ft
import warnings
warnings.filterwarnings('ignore')
import time

In [ ]:
pip install shap

In [ ]:
pip install featuretools

In [ ]:
pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 1. Check datasets

## 1.1 Read dataset

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/캐글 스터디 필사/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/캐글 스터디 필사/test.csv')

In [ ]:
print('df_train shape:', df_train.shape, '  ', 'df_test shape:', df_test.shape)

df_train shape: (12367, 143)    df_test shape: (23856, 142)


In [ ]:
df_train.head(5)

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,190000.0,0,3,0,1,1,0,NaN,0,...,100.0,1849.0,1.0,100.0,0.0,1.000000,0.0,100.0,1849.0,4.0
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,144.0,4489.0,1.0,144.0,0.0,1.000000,64.0,144.0,4489.0,4.0
2,ID_68de51c94,NaN,0,8,0,1,1,0,NaN,0,...,121.0,8464.0,1.0,0.0,0.0,0.250000,64.0,121.0,8464.0,4.0
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,81.0,289.0,16.0,121.0,4.0,1.777778,1.0,121.0,289.0,4.0
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,121.0,1369.0,16.0,121.0,4.0,1.777778,1.0,121.0,1369.0,4.0


In [ ]:
df_test.head(5)

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq
0,ID_2f6873615,NaN,0,5,0,1,1,0,NaN,1,...,4,0,16,9,0,1,2.25,0.25,272.25,16
1,ID_1c78846d2,NaN,0,5,0,1,1,0,NaN,1,...,41,256,1681,9,0,1,2.25,0.25,272.25,1681
2,ID_e5442cf6a,NaN,0,5,0,1,1,0,NaN,1,...,41,289,1681,9,0,1,2.25,0.25,272.25,1681
3,ID_a8db26a79,NaN,0,14,0,1,1,1,1.0,0,...,59,256,3481,1,256,0,1.00,0.00,256.00,3481
4,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,0,...,18,121,324,1,0,1,0.25,64.00,NaN,324


## 1.2 Make descriprtion df

In [ ]:
description = [
('v2a1', 'Monthly rent payment'),
('hacdor', '=1 Overcrowding by bedrooms'),
('rooms', 'number of all rooms in the house'),
('hacapo', '=1 Overcrowding by rooms'),
('v14a', '=1 has toilet in the household'),
('refrig', '=1 if the household has refrigerator'),
('v18q', 'owns a tablet'),
('v18q1', 'number of tablets household owns'),
('r4h1', 'Males younger than 12 years of age'),
('r3h2', 'Males 12 years of age and older'),
('r4h3', 'Total males in the household'),
("r4m1"," Females younger than 12 years of age"),
("r4m2"," Females 12 years of age and older"),
("r4m3"," Total females in the household"),
("r4t1"," persons younger than 12 years of age"),
("r4t2"," persons 12 years of age and older"),
("r4t3"," Total persons in the household"),
("tamhog"," size of the household"),
("tamviv"," number of persons living in the household"),
("escolari"," years of schooling"),
("rez_esc"," Years behind in school"),
("hhsize"," household size"),
("paredblolad"," =1 if predominant material on the outside wall is block or brick"),
("paredzocalo"," =1 if predominant material on the outside wall is socket (wood, zinc or absbesto"),
("paredpreb"," =1 if predominant material on the outside wall is prefabricated or cement"),
("pareddes"," =1 if predominant material on the outside wall is waste material"),
("paredmad"," =1 if predominant material on the outside wall is wood"),
("paredzinc"," =1 if predominant material on the outside wall is zink"),
("paredfibras"," =1 if predominant material on the outside wall is natural fibers"),
("paredother"," =1 if predominant material on the outside wall is other"),
("pisomoscer"," =1 if predominant material on the floor is mosaic ceramic terrazo"),
("pisocemento"," =1 if predominant material on the floor is cement"),
("pisoother"," =1 if predominant material on the floor is other"),
("pisonatur"," =1 if predominant material on the floor is  natural material"),
("pisonotiene"," =1 if no floor at the household"),
("pisomadera"," =1 if predominant material on the floor is wood"),
("techozinc"," =1 if predominant material on the roof is metal foil or zink"),
("techoentrepiso"," =1 if predominant material on the roof is fiber cement,   mezzanine "),
("techocane"," =1 if predominant material on the roof is natural fibers"),
("techootro"," =1 if predominant material on the roof is other"),
("cielorazo"," =1 if the house has ceiling"),
("abastaguadentro"," =1 if water provision inside the dwelling"),
("abastaguafuera"," =1 if water provision outside the dwelling"),
("abastaguano"," =1 if no water provision"),
("public"," =1 electricity from CNFL,  ICE, ESPH/JASEC"),
("planpri"," =1 electricity from private plant"),
("noelec"," =1 no electricity in the dwelling"),
("coopele"," =1 electricity from cooperative"),
("sanitario1"," =1 no toilet in the dwelling"),
("sanitario2"," =1 toilet connected to sewer or cesspool"),
("sanitario3"," =1 toilet connected to  septic tank"),
("sanitario5"," =1 toilet connected to black hole or letrine"),
("sanitario6"," =1 toilet connected to other system"),
("energcocinar1"," =1 no main source of energy used for cooking (no kitchen)"),
("energcocinar2"," =1 main source of energy used for cooking electricity"),
("energcocinar3"," =1 main source of energy used for cooking gas"),
("energcocinar4"," =1 main source of energy used for cooking wood charcoal"),
("elimbasu1"," =1 if rubbish disposal mainly by tanker truck"),
("elimbasu2"," =1 if rubbish disposal mainly by botan hollow or buried"),
("elimbasu3"," =1 if rubbish disposal mainly by burning"),
("elimbasu4"," =1 if rubbish disposal mainly by throwing in an unoccupied space"),
("elimbasu5"," =1 if rubbish disposal mainly by throwing in river,   creek or sea"),
("elimbasu6"," =1 if rubbish disposal mainly other"),
("epared1"," =1 if walls are bad"),
("epared2"," =1 if walls are regular"),
("epared3"," =1 if walls are good"),
("etecho1"," =1 if roof are bad"),
("etecho2"," =1 if roof are regular"),
("etecho3"," =1 if roof are good"),
("eviv1"," =1 if floor are bad"),
("eviv2"," =1 if floor are regular"),
("eviv3"," =1 if floor are good"),
("dis"," =1 if disable person"),
("male"," =1 if male"),
("female"," =1 if female"),
("estadocivil1"," =1 if less than 10 years old"),
("estadocivil2"," =1 if free or coupled union"),
("estadocivil3"," =1 if married"),
("estadocivil4"," =1 if divorced"),
("estadocivil5"," =1 if separated"),
("estadocivil6"," =1 if widower"), #홀아비
("estadocivil7"," =1 if single"),
("parentesco1"," =1 if household head"),
("parentesco2"," =1 if spouse/partner"),
("parentesco3"," =1 if son/doughter"),
("parentesco4"," =1 if stepson/doughter"),  #의붓아들/딸
("parentesco5"," =1 if son/doughter in law"),
("parentesco6"," =1 if grandson/doughter"),
("parentesco7"," =1 if mother/father"),
("parentesco8"," =1 if father/mother in law"),
("parentesco9"," =1 if brother/sister"),
("parentesco10"," =1 if brother/sister in law"),
("parentesco11"," =1 if other family member"),
("parentesco12"," =1 if other non family member"),
("idhogar"," Household level identifier"),
("hogar_nin"," Number of children 0 to 19 in household"),
("hogar_adul"," Number of adults in household"),
("hogar_mayor"," # of individuals 65+ in the household"),
("hogar_total"," # of total individuals in the household"), #("r4t3"," Total persons in the household")이랑 똑같은거 아닌가
("dependency"," Dependency rate"),
("edjefe"," years of education of male head of household"),
("edjefa"," years of education of female head of household"),
("meaneduc","average years of education for adults (18+)"),
("instlevel1"," =1 no level of education"),
("instlevel2"," =1 incomplete primary"),
("instlevel3"," =1 complete primary"),
("instlevel4"," =1 incomplete academic secondary level"),
("instlevel5"," =1 complete academic secondary level"),
("instlevel6"," =1 incomplete technical secondary level"),
("instlevel7"," =1 complete technical secondary level"),
("instlevel8"," =1 undergraduate and higher education"),
("instlevel9"," =1 postgraduate higher education"),
("bedrooms"," number of bedrooms"),
("overcrowding"," # persons per room"),
("tipovivi1"," =1 own and fully paid house"),    #자가이고 완납
("tipovivi2"," =1 own, paying in installments"), #집이 자가지만 할부로 납부
("tipovivi3"," =1 rented"),
("tipovivi4"," =1 precarious"),
("tipovivi5"," =1 other(assigned"),
("computer"," =1 if the household has notebook or desktop computer,   borrowed)"),
("television"," =1 if the household has TV"),
("mobilephone"," =1 if mobile phone"),
("qmobilephone"," # of mobile phones"),
("lugar1"," =1 region Central"),
("lugar2"," =1 region Chorotega"),
("lugar3"," =1 region PacÃƒÂ­fico central"),
("lugar4"," =1 region Brunca"),
("lugar5"," =1 region Huetar AtlÃƒÂ¡ntica"),
("lugar6"," =1 region Huetar Norte"),
("area1"," =1 zona urbana"),
("area2"," =2 zona rural"),
("age"," Age in years"),
("SQBescolari"," escolari squared"),
("SQBage"," age squared"),
("SQBhogar_total"," hogar_total squared"),
("SQBedjefe"," edjefe squared"),
("SQBhogar_nin"," hogar_nin squared"),
("SQBovercrowding"," overcrowding squared"),
("SQBdependency"," dependency squared"),
("SQBmeaned"," meaned squared"),
("agesq"," Age squared"),]

description = pd.DataFrame(description, columns=['varname', 'description'])


In [ ]:
description

## 1.3 Check null data

In [ ]:
Total = df_train.isnull().sum().sort_values(ascending=False)
Percent = 100 * (df_train.isnull().sum() / df_train.isnull().count()).sort_values(ascending=False)
missing_df = pd.concat([Total, Percent], axis=1, keys=['Total', 'Percent'])

missing_df.head(20)

,Total,Percent
rez_esc,10316,83.415541
v18q1,9474,76.607100
v2a1,8844,71.512897
SQBmeaned,7,0.056602
meaneduc,7,0.056602
Target,2,0.016172
instlevel9,2,0.016172
tipovivi5,2,0.016172
tipovivi4,2,0.016172
tipovivi3,2,0.016172


## 1.4 Fill missing values

In [ ]:
# if education is 'yes' and person in head of household, fill with escolari
# 가장이고, 교육기간이 있으면 escolari 컬럼 채울 수 있음
# "edjefe"," years of education of male head of household"
# "parentesco1"," =1 if household head"
# "escolari"," years of schooling"

#edjefa 가 yes 이고 parentescol이 1인 행 edgefa컬럼 값에 escolari값을 채우기
df_train.loc[(df_train['edjefa']=='yes') & (df_train['parentesco1'] == 1), 'edjefa'] = df_train.loc[(df_train['edjefa'] == 'yes') & (df_train['parentesco1'] == 1), 'escolari']
df_train.loc[(df_train['edjefe'] =='yes') & (df_train['parentesco1'] == 1), 'edjefe'] = df_train.loc[(df_train['edjefe'] == 'yes') & (df_train['parentesco1'] == 1), 'escolari']

df_test.loc[(df_test['edjefa'] == 'yes') & (df_test['parentesco1'] == 1), 'edjefa'] = df_test.loc[(df_test['edjefa'] == 'yes') & (df_test['parentesco1'] == 1), 'escolari']
df_test.loc[(df_test['edjefe'] == 'yes') & (df_test['parentesco1'] == 1), 'edjefe'] = df_test.loc[(df_test['edjefe'] == 'yes') & (df_test['parentesco1'] == 1), 'escolari']

#this field is supposed to be interaction between gender and escolari, but it isn't clear what 'yes' meas, lets fill it with 4
#왜 4를 채우는 거지??
df_train.loc[df_train['edjefa'] == 'yes', 'edjefa'] = 4
df_train.loc[df_train['edjefe'] == 'yes', 'edjefe'] = 4

df_test.loc[df_test['edjefa'] == 'yes', 'edjefa'] = 4
df_test.loc[df_test['edjefe'] == 'yes', 'edjefe'] = 4

#create feature with max education of either head of household
df_train['edjef'] = np.max(df_train[['edjefa', 'edjefe']], axis=1)
df_test['edjef'] = np.max(df_test[['edjefa', 'edjefe']], axis=1)

#fix some inconsistencies in the data - some rows indicate both that the household does and does not have toilet,
#if there is no water we'll assume they do not
# 화장실이 있고, 주거지에 화장실이 없고, 물 공급이 되지않은 경우 => 'v14a' 컬럼과 'sanitario1' 컬럼 모두 0으로 변환
df_train.loc[(df_train.v14a == 1) & (df_train.sanitario1 == 1) & (df_train.abastaguano == 0), 'v14a'] = 0
df_train.loc[(df_train.v14a == 1) & (df_train.sanitario1 == 1) & (df_train.abastaguano == 0), 'sanitario1'] = 0

df_test.loc[(df_test.v14a == 1) & (df_test.sanitario1 == 1) & (df_test.abastaguano == 0), 'v14a'] = 0
df_test.loc[(df_test.v14a == 1) & (df_test.sanitario1 == 1) & (df_test.abastaguano == 0), 'sanitario1'] = 0

### rez_esz, SQBmeande
- rez_esc : Years behind in school -> filled with 0
- SQBmeaned : square of the mean years of education of adults (>=18) in the household agesq, Age squared -> same with rez_esc -> filled with 0

In [ ]:
df_train['rez_esc'].fillna(0, inplace=True)
df_test['rez_esc'].fillna(0, inplace=True)

In [ ]:
df_train['SQBmeaned'].fillna(0, inplace=True)
df_test['SQBmeaned'].fillna(0, inplace=True)

### meaneduc
- meaneduc : average years of education for adults (18+) -> filled with 0

In [ ]:
df_train['meaneduc'].fillna(0, inplace=True)
df_test['meaneduc'].fillna(0, inplace=True)

### v18q1
- v18q : number of tablets household owns -> if v18q(Do you own a tablet?) == 1, there are some values. if not, only NaN values in v18q1. See below 3 cells

In [ ]:
df_train['v18q'].value_counts()

0    9474
1    2893
Name: v18q, dtype: int64

- v18q1 : number of tablets household owns -> if v18q == 1, there are some values. if not, only NaN values there. See below two cells

In [ ]:
df_train.loc[df_train['v18q']==1, 'v18q1'].value_counts()

1.0    2091
2.0     565
3.0     157
4.0      57
5.0      17
6.0       6
Name: v18q1, dtype: int64

In [ ]:
df_train.loc[df_train['v18q'] == 0, 'v18q1'].value_counts()

Series([], Name: v18q1, dtype: int64)

In [ ]:
df_train['v18q1'].fillna(0, inplace=True)
df_test['v18q'].fillna(0, inplace=True)

- v2a1: Monthly rent payment -> if tipovivi3(rented?) == 1, there are some values. If not there are also some values.
- NaN value could be replaced by 0

- 만약 렌트값인 tipovivi3이 1이라면 v2al값도 영향 있을 것 -> so, nan값은 0으로 대체하면됨

In [ ]:
df_train['tipovivi3'].value_counts()

0.0    10161
1.0     2204
Name: tipovivi3, dtype: int64

In [ ]:
#tipovivi3이 1일때와 0일때의 'v2a1'값을 kdeplot 으로 표현
sns.kdeplot(df_train.loc[df_train['tipovivi3'] == 1, 'v2a1'], label='Monthly rent payment of household(rented=1)')
sns.kdeplot(df_train.loc[df_train['tipovivi3'] == 0, 'v2a1'], label='Monthly rent payment of household(rented=0)')
plt.xscale('log')
plt.legend()
plt.show()
#lable값은 왜 안나오지? => plt.legend() 코드를 추가해야 레이블이 나옴

#tipovivi3이 1일때 monthly rent payment 중 높은 값이 밀도가 높음

In [ ]:
#v2al컬럼 중 null값을 0으로 대체
df_train['v2a1'].fillna(0, inplace=True)
df_test['v2a1'].fillna(0, inplace=True)


In [ ]:
#df_train null 값 다시 체크
Total = df_train.isnull().sum().sort_values(ascending=False)
Percent = 100 * (df_train.isnull().sum() / df_train.isnull().count()).sort_values(ascending=False)
missing_df = pd.concat([Total, Percent], axis=1, keys=['Total', 'Percent'])

missing_df.head(20)

,Total,Percent
edjef,12367,100.000000
bedrooms,2,0.016172
television,2,0.016172
computer,2,0.016172
tipovivi5,2,0.016172
tipovivi4,2,0.016172
tipovivi3,2,0.016172
tipovivi2,2,0.016172
tipovivi1,2,0.016172
overcrowding,2,0.016172


In [ ]:
#df_test null 값 체크
Total = df_test.isnull().sum().sort_values(ascending=False)
Percent = 100 * (df_test.isnull().sum() / df_test.isnull().count()).sort_values(ascending=False)
missing_df = pd.concat([Total, Percent], axis=1, keys=['Total', 'Percent'])

missing_df.head(20)

,Total,Percent
edjef,23856,100.000000
v18q1,18126,75.980885
hogar_mayor,0,0.000000
parentesco10,0,0.000000
parentesco11,0,0.000000
parentesco12,0,0.000000
idhogar,0,0.000000
hogar_nin,0,0.000000
hogar_adul,0,0.000000
hogar_total,0,0.000000


## For now, there are no NaN values
- 노트북데이터랑 내 데이터 왜 다르지?
- 'edjef'컬럼에 null값 아직 많은데 왜 없다고 판단하지? => 위에서 새로 만든 컬럼, edjefe, edjefa의 최댓값

In [ ]:
df_train['edjef'].value_counts()

Series([], Name: edjef, dtype: int64)

# 2. Feature engineering

## 2.1 Object features

In [ ]:
#데이터 타입이 object인 컬럼의 리스트인 features_object
features_object = [col for col in df_train.columns if df_train[col].dtype == 'object']

In [ ]:
features_object

['Id', 'idhogar', 'dependency', 'edjefe', 'edjefa']

In [ ]:
df_train['dependency']

0        0.0
1        8.0
2        8.0
3        1.0
4        1.0
        ... 
12362    2.5
12363    2.5
12364    2.5
12365    2.5
12366    NaN
Name: dependency, Length: 12367, dtype: float64

## dependecy

In [ ]:
# some dependencies are Na, fill those with the square root of the square
# 부양률에는 루트값을 취해줌

df_train['dependency'] = np.sqrt(df_train['SQBdependency'])
df_test['dependency'] = np.sqrt(df_test['SQBdependency'])

In [ ]:
# df_train['dependency'] = df_train['dependency'].replace({np.inf: 0})
# df_test['dependency'] = df_test['dependency'].replace({np.inf: 0})

# def replace_dependency(x):
#     if x == 'yes':
#         return 10
#     elif x == 'no':
#         return 0
#     else:
#         return x

# df_train['dependency'] = df_train['dependency'].apply(replace_dependency).astype(float)
# df_test['dependency'] = df_test['dependency'].apply(replace_dependency).astype(float)

# - As you can see, setting yes -> 10 and no -> 0 is good choice.
# - At first, fill inf value with 0.

In [ ]:
#df_train['edjefa']

0         no
1         no
2         11
3         no
4         no
        ... 
12362     no
12363     no
12364     no
12365     no
12366    NaN
Name: edjefa, Length: 12367, dtype: object

## edjefe
- edjefe, years of education of male head of household based on the interaction of escolari(years of education), 
- head of household and gender, yes=1 and no=0
- replace yes -> 1 and no -> 0

In [ ]:
#yes -> 1, no -> 0으로 변환할 함수 정의
def replace_edjefa(x):
  if x == 'yes':
    return 1
  elif x == 'no':
    return 0
  else:
    return x


df_train['edjefa'] = df_train['edjefa'].apply(replace_edjefa).astype(float)
df_test['edjefa'] = df_test['edjefa'].apply(replace_edjefa).astype(float)

In [ ]:
#create feature with max education of either head of household
#남성 여성 가장 중 더 max education year을 'edjef' 컬럼으로

df_train['edjef'] = np.max(df_train[['edjefa', 'edjefe']], axis=1)
df_test['edjef'] = np.max(df_test[['edjefa', 'edjefe']], axis=1)

## roof and electricity

In [ ]:
#새로운 컬럼 'roof_waste_material' 과 'electricity_other' 생성, nan값으로 채위
df_train['roof_waste_material'] = np.nan
df_test['roof_waste_material'] = np.nan
df_train['electricity_other'] = np.nan
df_test['electricity_other'] = np.nan

def fill_roof_exception(x):
  if (x['techozinc'] == 0) and (x['techoentrepiso'] == 0) and (x['techocane'] == 0) and (x['techootro'] == 0):
    return 1
  else:
    return 0

def fill_no_electricity(x):
  if (x['public'] == 0) and (x['planpri'] == 0) and (x['noelec'] == 0) and (x['coopele'] == 0):
    return 1
  else:
    return 0

df_train['roof_waste_material'] = df_train.apply(lambda x : fill_roof_exception(x), axis=1)
df_test['roof_waste_material'] = df_test.apply(lambda x : fill_roof_exception(x), axis=1)
df_train['electricity_other'] = df_train.apply(lambda x : fill_no_electricity(x), axis=1)
df_test['electricity_other'] = df_test.apply(lambda x : fill_no_electricity(x), axis=1)



## 2.2 Extract cat features
- According to data scription, there are many binary category features.

In [ ]:
#binary 한 값을 가지는 컬럼의 리스트 생성
binary_cat_features = [col for col in df_train.columns if df_train[col].value_counts().shape[0] == 2]

## 2.3 Make new features using continuous feature

In [ ]:
#연속 피처 리스트생성
#전체 컬럼 중 이진형 features를 제외한 리스트 생성
#object feature를 제외한 버전으로 업데이트
#id, target, idhogar같은 키값을 제외한 리스트로 업데이트

#위의 리스트 말고는 연속데이터로 봄
continuous_features = [col for col in df_train.columns if col not in binary_cat_features]
continuous_features = [col for col in continuous_features if col not in features_object]
continuous_features = [col for col in continuous_features if col not in ['Id', 'Target', 'idhogar']]

In [ ]:
print('There are {} continuous features'.format(len(continuous_features)))
for col in continuous_features:
  print('{} : {}'.format(col, description.loc[description['varname'] == col, 'description'].values))


There are 41 continuous features
v2a1 : ['Monthly rent payment']
rooms : ['number of all rooms in the house']
v18q1 : ['number of tablets household owns']
r4h1 : ['Males younger than 12 years of age']
r4h2 : []
r4h3 : ['Total males in the household']
r4m1 : [' Females younger than 12 years of age']
r4m2 : [' Females 12 years of age and older']
r4m3 : [' Total females in the household']
r4t1 : [' persons younger than 12 years of age']
r4t2 : [' persons 12 years of age and older']
r4t3 : [' Total persons in the household']
tamhog : [' size of the household']
tamviv : [' number of persons living in the household']
escolari : [' years of schooling']
rez_esc : [' Years behind in school']
hhsize : [' household size']
elimbasu5 : [' =1 if rubbish disposal mainly by throwing in river,   creek or sea']
parentesco9 : [' =1 if brother/sister']
parentesco10 : [' =1 if brother/sister in law']
parentesco11 : [' =1 if other family member']
parentesco12 : [' =1 if other non family member']
hogar_nin :

- hhsize : household size
- tamhog : size of the household

What is different

- As you can see, the meaning of two features are same but the exact number are different. Are they different?
- I don't know. For now, I decided to drop one feature 'tamhog'

In [ ]:
df_train['edjef'].value_counts()

0.0     8108
6.0     1199
11.0     493
9.0      310
8.0      282
15.0     246
4.0      235
5.0      235
7.0      225
3.0      213
14.0     159
16.0     139
10.0     120
12.0      97
17.0      92
2.0       90
13.0      70
1.0       35
18.0       6
21.0       5
19.0       4
20.0       2
Name: edjef, dtype: int64

In [ ]:
df_train.drop('tamhog', axis=1, inplace=True)
df_test.drop('tamhog', axis=1, inplace=True)

## Squared features
- There are many squared features. Actually, tree models like lightgbm don't need them.  => 분류일때는 squared값 필요 없음
- But at this kernel, I want to use lightgbm as feature filter model and set entity- embedding as classfier. So Let's keep them.  ?????

## Family features
- hogar_nin, hogar_adul, hogar_mayor, hogar_total, r4h1, r4h2, r4h3, r4m1, r4m2, r4m3, r4t1, r4t2, r4t3, tmbhog, tamvid, rez_esc, escolari

- Family size features (substract, ratio)

In [ ]:
##df_train data

df_train['adult'] = df_train['hogar_adul'] - df_train['hogar_mayor']
#dependency 관련
df_train['dependency_count'] = df_train['hogar_nin'] + df_train['hogar_mayor']
df_train['dependency'] = df_train['dependency_count'] / df_train['adult']
#전체 연령별
df_train['child_percent'] = df_train['hogar_nin'] / df_train['hogar_total']
df_train['elder_percent'] = df_train['hogar_mayor'] / df_train['hogar_total']
df_train['adult_percent'] = df_train['hogar_adul'] / df_train['hogar_total']
#남성 12세 이하, 12세 이상, 전체 비율
df_train['males_younger_12_years_percent'] = df_train['r4h1'] / df_train['hogar_total']
df_train['males_older_12_years_percent'] = df_train['r4h2'] / df_train['hogar_total']
df_train['males_percent'] = df_train['r4h3'] / df_train['hogar_total']
#여성 12세 이하, 12세 이상, 전체 비율
df_train['females_younger_12_years_percent'] = df_train['r4m1'] / df_train['hogar_total']
df_train['females_older_12_years_percent'] = df_train['r4m2'] / df_train['hogar_total']
df_train['females_percent'] = df_train['r4m3'] / df_train['hogar_total']
#전체 중 12세 이상, 이하, 전체 ?? 
#"hogar_total"," # of total individuals in the household"
#"r4t3"," Total persons in the household"
df_train['persons_younger_12_years_percent'] = df_train['r4t1'] / df_train['hogar_total']
df_train['persons_older_12_years_percent'] = df_train['r4t2'] / df_train['hogar_total']
df_train['persons_percent'] = df_train['r4t3'] / df_train['hogar_total']


- family ratio : ~~~~ / "hogar_total"," # of total individuals in the household"

In [ ]:
##df_test data

df_test['adult'] = df_test['hogar_adul'] - df_test['hogar_mayor']
#dependency 관련
df_test['dependency_count'] = df_test['hogar_nin'] + df_test['hogar_mayor']
df_test['dependency'] = df_test['dependency_count'] / df_test['adult']
#전체 연령별
df_test['child_percent'] = df_test['hogar_nin'] / df_test['hogar_total']
df_test['elder_percent'] = df_test['hogar_mayor'] / df_test['hogar_total']
df_test['adult_percent'] = df_test['hogar_adul'] / df_test['hogar_total']
#남성 12세 이하, 12세 이상, 전체 비율
df_test['males_younger_12_years_percent'] = df_test['r4h1'] / df_test['hogar_total']
df_test['males_older_12_years_percent'] = df_test['r4h2'] / df_test['hogar_total']
df_test['males_percent'] = df_test['r4h3'] / df_test['hogar_total']
#여성 12세 이하, 12세 이상, 전체 비율
df_test['females_younger_12_years_percent'] = df_test['r4m1'] / df_test['hogar_total']
df_test['females_older_12_years_percent'] = df_test['r4m2'] / df_test['hogar_total']
df_test['females_percent'] = df_test['r4m3'] / df_test['hogar_total']
#전체 중 12세 이상, 이하, 전체 ?? 
#"hogar_total"," # of total individuals in the household"
#"r4t3"," Total persons in the household"
df_test['persons_younger_12_years_percent'] = df_test['r4t1'] / df_test['hogar_total']
df_test['persons_older_12_years_percent'] = df_test['r4t2'] / df_test['hogar_total']
df_test['persons_percent'] = df_test['r4t3'] / df_test['hogar_total']


- family size : ~~~ / 'hhsize' : household size

In [ ]:
##df_train data

#male
df_train['males_younger_12_years_in_household_size'] = df_train['r4h1'] / df_train['hhsize']
df_train['males_older_12_years_in_household_size'] = df_train['r4h2'] / df_train['hhsize']
df_train['males_in_household_size'] = df_train['r4h3'] / df_train['hhsize']
#female
df_train['females_younger_12_years_in_household_size'] = df_train['r4m1'] / df_train['hhsize']
df_train['females_older_12_years_in_household_size'] = df_train['r4m2'] / df_train['hhsize']
df_train['females_in_household_size'] = df_train['r4m3'] / df_train['hhsize']  #노트북에는 'hogar_total로 되어있음
#person
df_train['persons_younger_12_years_in_household_size'] = df_train['r4t1'] / df_train['hhsize']
df_train['persons_older_12_years_in_household_size'] = df_train['r4t2'] / df_train['hhsize']
df_train['persons_in_household_size'] = df_train['r4t3'] / df_train['hhsize']

In [ ]:
##df_test data

#male
df_test['males_younger_12_years_in_household_size'] = df_test['r4h1'] / df_test['hhsize']
df_test['males_older_12_years_in_household_size'] = df_test['r4h2'] / df_test['hhsize']
df_test['males_in_household_size'] = df_test['r4h3'] / df_test['hhsize']
#female
df_test['females_younger_12_years_in_household_size'] = df_test['r4m1'] / df_test['hhsize']
df_test['females_older_12_years_in_household_size'] = df_test['r4m2'] / df_test['hhsize']
df_test['females_in_household_size'] = df_test['r4m3'] / df_test['hhsize']  #노트북에는 'hogar_total로 되어있음
#person
df_test['persons_younger_12_years_in_household_size'] = df_test['r4t1'] / df_test['hhsize']
df_test['persons_older_12_years_in_household_size'] = df_test['r4t2'] / df_test['hhsize']
df_test['persons_in_household_size'] = df_test['r4t3'] / df_test['hhsize']

In [ ]:
#('hacdor', '=1 Overcrowding by bedrooms'), ('hacapo', '=1 Overcrowding by rooms')
# 침실과 방에 overcrowding 평균치 컬럼 생성

df_train['overcrowding_room_and_bedroom'] = (df_train['hacdor'] + df_train['hacapo']) / 2
df_test['overcrowding_room_and_bedroom'] = (df_test['hacdor'] + df_test['hacapo']) / 2 

In [ ]:
#"escolari"," years of schooling"
#"age"," Age in years"

#재학기간 / 나이 비율
df_train['escolari_age'] = df_train['escolari']/ df_train['age']
df_test['escolari_age'] = df_test['escolari']/ df_test['age']

#"hogar_nin"," Number of children 0 to 19 in household"
#"r4t1"," persons younger than 12 years of age"
#나이가 12~19사이인 수
df_train['age_12_19'] = df_train['hogar_nin'] - df_train['r4t1']
df_test['age_12_19'] = df_test['hogar_nin'] - df_test['r4t1']


In [ ]:
#"tamviv"," number of persons living in the household"
#"qmobilephone"," # of mobile phones"
#'v18q1', 'number of tablets household owns'
#'v2a1', 'Monthly rent payment'

#사람당 핸드폰 수, 태블릿 수, 방 수, rent payment 비율
df_train['phones-per-capita'] = df_train['qmobilephone'] / df_train['tamviv']
df_train['tablets-per-capita'] = df_train['v18q1'] / df_train['tamviv']
df_train['rooms-per-capita'] = df_train['rooms'] / df_train['tamviv']
df_train['rent-per-capita'] = df_train['v2a1'] / df_train['tamviv']

In [ ]:
df_test['phones-per-capita'] = df_test['qmobilephone'] / df_test['tamviv']
df_test['tablets-per-capita'] = df_test['v18q1'] / df_test['tamviv']
df_test['rooms-per-capita'] = df_test['rooms'] / df_test['tamviv']
df_test['rent-per-capita'] = df_test['v2a1'] / df_test['tamviv']

- You can see that "Total persons in the household" != "# of total individuals in the household".
- Somewhat weired. But for now I will keep it.

In [ ]:
#"r4t3"," Total persons in the household"
(df_train['hogar_total'] == df_train['r4t3']).sum()

12307

## Rent per family features
- I will reduce the number of features using shap
- so let's generate many features!! 
- Hope catch some fortune features :)


In [ ]:
family_size_features = ['adult', 'hogar_adul', 'hogar_mayor', 'hogar_nin', 
                        'hogar_total', 'r4h1', 'r4h2', 'r4h3', 'r4m1', 'r4m2', 
                        'r4m3', 'r4t1', 'r4t2', 'r4t3','hhsize']

new_feats = []

#'v2a1', 'Monthly rent payment'

for col in family_size_features:
  new_col_name = 'new_{}_per_{}'.format('v2a1', col)
  new_feats.append(new_col_name)
  df_train[new_col_name] = df_train['v2a1'] / df_train[col]
  df_test[new_col_name] = df_test['v2a1'] / df_test[col]


- Ratio feature can have infinite values. So Let them be filled with 0

In [ ]:
for col in new_feats:
  df_train[col].replace([np.inf], np.nan, inplace=True)
  df_train[col].fillna(0, inplace=True)

  df_test[col].replace([np.inf], np.nan, inplace=True)
  df_test[col].fillna(0, inplace=True)

## Room per family features

In [ ]:
new_feats = []

for col in family_size_features:
  new_col_name = 'new_{}_per{}'.format('rooms', col)
  new_feats.append(new_col_name)
  df_train[new_col_name] = df_train['rooms'] / df_train[col]
  df_test[new_col_name] = df_test['rooms'] / df_test[col]

for col in new_feats:
  df_train[col].replace([np.inf], np.nan, inplace=True)
  df_train[col].fillna(0, inplace=True)

  df_test[col].replace([np.inf], np.nan, inplace=True)
  df_test[col].fillna(0, inplace=True)

## BedRoom per family features

In [ ]:
new_feats = []

for col in family_size_features:
  new_col_name = 'new_{}_per{}'.format('bedrooms', col)
  new_feats.append(new_col_name)
  df_train[new_col_name] = df_train['bedrooms'] / df_train[col]
  df_test[new_col_name] = df_test['bedrooms'] / df_test[col]

for col in new_feats:
  df_train[col].replace([np.inf], np.nan, inplace=True)
  df_train[col].fillna(0, inplace=True)

  df_test[col].replace([np.inf], np.nan, inplace=True)
  df_test[col].fillna(0, inplace=True)

In [ ]:
#train data 와 test data간 피처 개수 체크
print(df_train.shape, df_test.shape)

(12367, 220) (23856, 219)


## Tabulet per family features

In [ ]:
new_feats = []

for col in family_size_features:
  new_col_name = 'new_{}_per{}'.format('v18q1', col)
  new_feats.append(new_col_name)
  df_train[new_col_name] = df_train['v18q1'] / df_train[col]
  df_test[new_col_name] = df_test['v18q1'] / df_test[col]

for col in new_feats:
  df_train[col].replace([np.inf], np.nan, inplace=True)
  df_train[col].fillna(0, inplace=True)

  df_test[col].replace([np.inf], np.nan, inplace=True)
  df_test[col].fillna(0, inplace=True)

## phone per family features

In [ ]:
new_feats = []

for col in family_size_features:
  new_col_name = 'new_{}_per{}'.format('qmobilephone', col)
  new_feats.append(new_col_name)
  df_train[new_col_name] = df_train['qmobilephone'] / df_train[col]
  df_test[new_col_name] = df_test['qmobilephone'] / df_test[col]

for col in new_feats:
  df_train[col].replace([np.inf], np.nan, inplace=True)
  df_train[col].fillna(0, inplace=True)

  df_test[col].replace([np.inf], np.nan, inplace=True)
  df_test[col].fillna(0, inplace=True)

## rez_esc(Years behind in school) per family features

In [ ]:
new_feats = []

for col in family_size_features:
  new_col_name = 'new_{}_per{}'.format('rez_esc', col)
  new_feats.append(new_col_name)
  df_train[new_col_name] = df_train['rez_esc'] / df_train[col]
  df_test[new_col_name] = df_test['rez_esc'] / df_test[col]

for col in new_feats:
  df_train[col].replace([np.inf], np.nan, inplace=True)
  df_train[col].fillna(0, inplace=True)

  df_test[col].replace([np.inf], np.nan, inplace=True)
  df_test[col].fillna(0, inplace=True)

In [ ]:
df_train['rez_esc_age'] = df_train['rez_esc'] / df_train['age']
df_train['rez_esc_escolari'] = df_train['rez_esc'] / df_train['escolari']

df_test['rez_esc_age'] = df_test['rez_esc'] / df_test['age']
df_test['rez_esc_escolari'] = df_test['rez_esc'] / df_test['escolari']

## Rich features
- I think the more richer, the larger number of phones and tabulet

In [ ]:
df_train['tabulet_x_qmobilephone'] = df_train['v18q1'] * df_train['qmobilephone']
df_test['tabulet_x_qmobilephone'] = df_test['v18q1'] * df_test['qmobilephone']


- wall, roof, floor may be key factor.
- Let's multiply each of them. 
- Becuase they are binary cat features, 
- so mulitification of each features generates new categorical features

In [ ]:
#("epared1"," =1 if walls are bad"),
#("epared2"," =1 if walls are regular"),
#("epared3"," =1 if walls are good"),
#("etecho1"," =1 if roof are bad"),
#("etecho2"," =1 if roof are regular"),
#("etecho3"," =1 if roof are good"),
#("eviv1"," =1 if floor are bad"),
#("eviv2"," =1 if floor are regular"),
#("eviv3"," =1 if floor are good"),

#wall and roof
for col1 in ['epared1', 'epared2', 'epared3']:
  for col2 in ['etecho1', 'etecho2', 'etecho3']:
    new_col_name = 'new_{}_x_{}'.format(col1, col2)
    df_train[new_col_name] = df_train[col1] * df_train[col2]
    df_test[new_col_name] = df_test[col1] * df_test[col2]

#wall and floor
for col1 in ['epared1', 'epared2', 'epared3']:
  for col2 in ['eviv1', 'eviv2', 'eviv3']:
    new_col_name = 'new_{}_x_{}'.format(col1, col2)
    df_train[new_col_name] = df_train[col1] * df_train[col2]
    df_test[new_col_name] = df_test[col1] * df_test[col2]

#roof and floor
for col1 in ['etecho1', 'etecho2', 'etecho3']:
  for col2 in ['eviv1', 'eviv2', 'eviv3']:
    new_col_name = 'new_{}_x_{}'.format(col1, col2)
    df_train[new_col_name] = df_train[col1] * df_train[col2]
    df_test[new_col_name] = df_test[col1] * df_test[col2]

- combination using three features

In [ ]:
for col1 in ['epared1', 'epared2', 'epared3']:
  for col2 in ['etecho1', 'etecho2', 'etecho3']:
    for col3 in ['eviv1', 'eviv2', 'eviv3']:
      new_col_name = 'new_{}_x_{}_x_{}'.format(col1, col2, col3)
      df_train[new_col_name] = df_train[col1] * df_train[col2] * df_train[col3]
      df_test[new_col_name] = df_test[col1] * df_test[col2] * df_test[col3]

In [ ]:
print(df_train.shape, df_test.shape)

(12367, 307) (23856, 306)


- I want to mix electricity and energy features -> energy features


In [ ]:
for col1 in ['public', 'planpri', 'noelec', 'coopele']:
  for col2 in ['energcocinar1', 'energcocinar2', 'energcocinar3', 'energcocinar4']:
    new_col_name = 'new_{}_x_{}'.format(col1, col2)
    df_train[new_col_name] = df_train[col1] * df_train[col2]
    df_test[new_col_name] = df_test[col1] * df_test[col2]

- I want to mix toilet and rubbish disposal features -> other_infra features


In [ ]:
for col1 in ['sanitario1', 'sanitario2', 'sanitario3', 'sanitario5', 'sanitario6']:
  for col2 in ['elimbasu1', 'elimbasu2', 'elimbasu3', 'elimbasu4', 'elimbasu5', 'elimbasu6']:
    new_col_name = 'new_{}_x_{}'.format(col1, col2)
    df_train[new_col_name] = df_train[col1] * df_train[col2]
    df_test[new_col_name] = df_test[col1] * df_test[col2]

- I want to mix toilet and water provision features -> water features


In [ ]:
for col1 in ['abastaguadentro', 'abastaguafuera', 'abastaguano']:
  for col2 in ['sanitario1', 'sanitario2', 'sanitario3', 'sanitario5', 'sanitario6']:
    new_col_name = 'new_{}_x_{}'.format(col1, col2)
    df_train[new_col_name] = df_train[col1] * df_train[col2]
    df_test[new_col_name] = df_test[col1] * df_test[col2]

In [ ]:
print(df_train.shape, df_test.shape)

(12367, 368) (23856, 367)


- I want mix education and area features -> education_zone_features
- ("instlevel1"," =1 no level of education"),
- ("instlevel2"," =1 incomplete primary"),
- ("instlevel3"," =1 complete primary"),
- ("instlevel4"," =1 incomplete academic secondary level"),
- ("instlevel5"," =1 complete academic secondary level"),
- ("instlevel6"," =1 incomplete technical secondary level"),
- ("instlevel7"," =1 complete technical secondary level"),
- ("instlevel8"," =1 undergraduate and higher education"),
- ("instlevel9"," =1 postgraduate higher education"),


In [ ]:
for col1 in ['area1', 'area2']:
  for col2 in ['instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9']:
    new_col_name = 'new_{}_x_{}'.format(col1, col2)
    df_train[new_col_name] = df_train[col1] * df_train[col2]
    df_test[new_col_name] = df_test[col1] * df_test[col2]

- Mix region and education


In [ ]:
for col1 in ['lugar1', 'lugar2', 'lugar3', 'lugar4', 'lugar5', 'lugar6']:
  for col2 in ['instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9']:
    new_col_name = 'new_{}_x_{}'.format(col1, col2)
    df_train[new_col_name] = df_train[col1] * df_train[col2]
    df_test[new_col_name] = df_test[col1] * df_test[col2]

In [ ]:
print(df_train.shape, df_test.shape)

(12367, 440) (23856, 439)


- Multiply television / mobilephone / computer / tabulet / refrigerator -> electornics features


In [ ]:
df_train['electronics'] = df_train['computer'] * df_train['mobilephone'] * df_train['television'] * df_train['v18q'] * df_train['refrig']
df_test['electronics'] = df_test['computer'] * df_test['mobilephone'] * df_test['television'] * df_test['v18q'] * df_test['refrig']

#덧셈? 곱셈?
df_train['no_appliances'] = df_train['refrig'] + df_train['computer'] + df_train['television'] + df_train['mobilephone']
df_test['no_appliances'] = df_test['refrig'] + df_test['computer'] + df_test['television'] + df_test['mobilephone']


- Mix wall material of roof, floor, wall

In [ ]:
## wall
#("paredblolad"," =1 if predominant material on the outside wall is block or brick"),
#("paredzocalo"," =1 if predominant material on the outside wall is socket (wood, zinc or absbesto"),
#("paredpreb"," =1 if predominant material on the outside wall is prefabricated or cement"),
#("pareddes"," =1 if predominant material on the outside wall is waste material"),
#("paredmad"," =1 if predominant material on the outside wall is wood"),
#("paredzinc"," =1 if predominant material on the outside wall is zink"),
#("paredfibras"," =1 if predominant material on the outside wall is natural fibers"),
#("paredother"," =1 if predominant material on the outside wall is other"),

## floor
#("pisomoscer"," =1 if predominant material on the floor is mosaic ceramic terrazo"),
#("pisocemento"," =1 if predominant material on the floor is cement"),
#("pisoother"," =1 if predominant material on the floor is other"),
#("pisonatur"," =1 if predominant material on the floor is  natural material"),
#("pisonotiene"," =1 if no floor at the household"),
#("pisomadera"," =1 if predominant material on the floor is wood"),

## roof
#("techozinc"," =1 if predominant material on the roof is metal foil or zink"),
#("techoentrepiso"," =1 if predominant material on the roof is fiber cement,   mezzanine "),
#("techocane"," =1 if predominant material on the roof is natural fibers"),
#("techootro"," =1 if predominant material on the roof is other"),

#wall, floor
for col1 in ['paredblolad', 'paredzocalo', 'paredpreb', 'pareddes', 'paredmad', 'paredzinc', 'paredfibras', 'paredother']:
  for col2 in ['pisomoscer', 'pisocemento', 'pisoother', 'pisonatur', 'pisonotiene', 'pisomadera']:
    new_col_name = 'new_{}_x_{}'.format(col1, col2)
    df_train[new_col_name] = df_train[col1] * df_train[col2]
    df_test[new_col_name] = df_test[col1] * df_test[col2]
#floor, roof
for col1 in ['pisomoscer', 'pisocemento', 'pisoother', 'pisonatur', 'pisonotiene', 'pisomadera']:
    for col1 in ['techozinc', 'techoentrepiso', 'techocane', 'techootro']:
        new_col_name = 'new_{}_x_{}'.format(col1, col2)
        df_train[new_col_name] = df_train[col1] * df_train[col2]
        df_test[new_col_name] = df_test[col1] * df_test[col2]
#wall, roof
for col1 in ['paredblolad', 'paredzocalo', 'paredpreb', 'pareddes', 'paredmad', 'paredzinc', 'paredfibras', 'paredother']:
    for col2 in ['techozinc', 'techoentrepiso', 'techocane', 'techootro']:
        new_col_name = 'new_{}_x_{}'.format(col1, col2)
        df_train[new_col_name] = df_train[col1] * df_train[col2]
        df_test[new_col_name] = df_test[col1] * df_test[col2]        
#wall, floor, roof    
for col1 in ['paredblolad', 'paredzocalo', 'paredpreb', 'pareddes', 'paredmad', 'paredzinc', 'paredfibras', 'paredother']:
    for col2 in ['pisomoscer', 'pisocemento', 'pisoother', 'pisonatur', 'pisonotiene', 'pisomadera']:
        for col3 in ['techozinc', 'techoentrepiso', 'techocane', 'techootro']:
            new_col_name = 'new_{}_x_{}_x_{}'.format(col1, col2, col3)
            df_train[new_col_name] = df_train[col1] * df_train[col2] * df_train[col3]
            df_test[new_col_name] = df_test[col1] * df_test[col2] * df_train[col3]


In [ ]:
df_train.shape, df_test.shape

((12367, 718), (23856, 716))

- Wow! without any aggreation features, we hvae 446 features for now. Actually mixing the materials of walls make thousands of features. I don't want to do that because of computational costs!


## Remove feature with only one value

In [ ]:
cols_with_only_one_value = []

for col in df_train.columns:
  if col == 'Target':
    continue
  if df_train[col].value_counts().shape[0] == 1 or df_test[col].value_counts().shape[0] == 1:
    print(col)
    cols_with_only_one_value.append(col)

elimbasu5
new_planpri_x_energcocinar1
new_planpri_x_energcocinar2
new_planpri_x_energcocinar3
new_planpri_x_energcocinar4
new_noelec_x_energcocinar2
new_sanitario1_x_elimbasu4
new_sanitario1_x_elimbasu5
new_sanitario1_x_elimbasu6
new_sanitario2_x_elimbasu4
new_sanitario2_x_elimbasu5
new_sanitario2_x_elimbasu6
new_sanitario3_x_elimbasu5
new_sanitario5_x_elimbasu4
new_sanitario5_x_elimbasu5
new_sanitario5_x_elimbasu6
new_sanitario6_x_elimbasu2
new_sanitario6_x_elimbasu4
new_sanitario6_x_elimbasu5
new_sanitario6_x_elimbasu6
new_abastaguafuera_x_sanitario6
new_abastaguano_x_sanitario2
new_abastaguano_x_sanitario6
new_paredblolad_x_pisonatur
new_paredblolad_x_pisonotiene
new_paredzocalo_x_pisoother
new_paredzocalo_x_pisonatur
new_paredpreb_x_pisonatur
new_pareddes_x_pisoother
new_pareddes_x_pisonatur
new_paredmad_x_pisoother
new_paredmad_x_pisonatur
new_paredzinc_x_pisoother
new_paredzinc_x_pisonatur
new_paredfibras_x_pisoother
new_paredfibras_x_pisonatur
new_paredfibras_x_pisonotiene
new_p

- Let's remove them!
- 값이 하나만 있는 컬럼

In [ ]:
df_train.drop(cols_with_only_one_value, axis=1, inplace=True)
df_test.drop(cols_with_only_one_value, axis=1, inplace=True)

## Check whether both train and test have same features


In [ ]:
cols_train = np.array(sorted([col for col in df_train.columns if col != 'Target']))
cols_test = np.array(sorted(df_test.columns))

In [ ]:
(cols_train == cols_test).sum() == len(cols_train)

True

## Aggregation for family features


In [ ]:
def max_min(x):
  return x.max() - x.min()

In [ ]:
family_size_features

['adult',
 'hogar_adul',
 'hogar_mayor',
 'hogar_nin',
 'hogar_total',
 'r4h1',
 'r4h2',
 'r4h3',
 'r4m1',
 'r4m2',
 'r4m3',
 'r4t1',
 'r4t2',
 'r4t3',
 'hhsize']

In [ ]:
agg_train = pd.DataFrame()
agg_test = pd.DataFrame()

for item in tqdm(family_size_features):
  for i, function in enumerate(['mean', 'std', 'min', 'max', 'sum', 'count', max_min]):
    # "idhogar"," Household level identifier"
    group_train = df_train[item].groupby(df_train['idhogar']).agg(function)
    group_test = df_test[item].groupby(df_train['idhogar']).agg(function)
    if i == 6:
      new_col = item + '_new_' + 'max_min'
    else:
      new_col = item + '_new_' + function
    agg_train[new_col] = group_train
    agg_test[new_col] = group_test

print('new aggregate train set has {} rows, and {} features'.format(agg_train.shape[0], agg_train.shape[1]))
print('new aggregate test set has {} rows, and {} features'.format(agg_test.shape[0], agg_test.shape[1]))



100%|██████████| 15/15 [00:08<00:00,  1.67it/s]

new aggregate train set has 2901 rows, and 105 features
new aggregate test set has 2901 rows, and 105 features


In [ ]:
aggr_list = ['rez_esc', 'dis', 'male', 'female', 'estadocivil1', 'estadocivil2', 'estadocivil3', 'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7', 
                  'parentesco2', 'parentesco3', 'parentesco4', 'parentesco5', 'parentesco6', 'parentesco7', 'parentesco8', 'parentesco9', 'parentesco10', 
                  'parentesco11', 'parentesco12','instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9',
                 'epared1', 'epared2', 'epared3', 'etecho1', 'etecho2', 'etecho3', 'eviv1', 'eviv2', 'eviv3', 'refrig', 'television', 'mobilephone',
            'area1', 'area2', 'v18q', 'edjef']


In [ ]:
for item in tqdm(aggr_list):
  for function in ['count', 'sum']:
    group_train = df_train[item].groupby(df_train['idhogar']).agg(function)
    group_test = df_test[item].groupby(df_test['idhogar']).agg(function)
    
    new_col = item + '_new1_' + function
    agg_train[new_col] = group_train
    agg_test[new_col] = group_test

print('new aggregate train set has {} rows, and {} features'.format(agg_train.shape[0], agg_train.shape[1]))
print('new aggregate test set has {} rows, and {} features'.format(agg_test.shape[0], agg_test.shape[1]))

100%|██████████| 47/47 [00:01<00:00, 32.51it/s]

new aggregate train set has 2901 rows, and 199 features
new aggregate test set has 2901 rows, and 199 features


In [ ]:
aggr_list = ['escolari', 'age', 'escolari_age', 'dependency', 'bedrooms', 'overcrowding', 'rooms', 'qmobilephone', 'v18q1']

for item in tqdm(aggr_list):
    for function in ['mean','std','min','max','sum', 'count', max_min]:
        group_train = df_train[item].groupby(df_train['idhogar']).agg(function)
        group_test = df_test[item].groupby(df_test['idhogar']).agg(function)
        if i == 6:
            new_col = item + '_new2_' + 'max_min'
        else:
            new_col = item + '_new2_' + function
        agg_train[new_col] = group_train
        agg_test[new_col] = group_test

print('new aggregate train set has {} rows, and {} features'.format(agg_train.shape[0], agg_train.shape[1]))
print('new aggregate test set has {} rows, and {} features'.format(agg_test.shape[0], agg_test.shape[1]))


100%|██████████| 9/9 [00:17<00:00,  1.96s/it]

new aggregate train set has 2901 rows, and 208 features
new aggregate test set has 2901 rows, and 208 features


In [ ]:
agg_test = agg_test.reset_index()
agg_train = agg_train.reset_index()

train_agg = pd.merge(df_train, agg_train, on='idhogar')
test = pd.merge(df_test, agg_test, on='idhogar')

#fill all na as 0
train_agg.fillna(value=0, inplace=True)
test.fillna(value=0, inplace=True)

print('train shape:', train_agg.shape, 'test shape:', test.shape)

train shape: (12367, 710) test shape: (0, 709)


In [ ]:
aggr_list = ['rez_esc', 'dis', 'male', 'female', 
                  'estadocivil1', 'estadocivil2', 'estadocivil3', 'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7', 
                  'parentesco2', 'parentesco3', 'parentesco4', 'parentesco5', 'parentesco6', 'parentesco7', 'parentesco8', 'parentesco9', 'parentesco10', 
                  'parentesco11', 'parentesco12',
                  'instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9',
                 'epared1', 'epared2', 'epared3', 'etecho1', 'etecho2', 'etecho3', 'eviv1', 'eviv2', 'eviv3', 'refrig', 'television', 'mobilephone',
            'area1', 'area2', 'v18q', 'edjef']


In [ ]:
for lugar in ['lugar1', 'lugar2', 'lugar3', 'lugar4', 'lugar5', 'lugar6']:
  group_train = df_train[[lugar, 'idhogar'] + aggr_list].groupby([lugar, 'idhogar']).sum().rest_index()
